*  Before starting, click "Runtime" in the top panel, select "Change runtime type" and then choose "GPU"

*  This tutorial follows the mtlsd tutorial, and is therefore condensed. Check out the mtlsd tutorial (**train_mtlsd.ipynb**) if there is any confusion throughout

*  Try running each cell consecutively to see what is happening before changing things around

*  Some cells are collapsed by default, these are generally utility functions or are expanded by defaullt in a previous tutorial. Double click to expand/collapse

*  sometimes colab can be slow, if this happens you may need to restart the runtime. also, you generally can only run one session at a time.

In [ ]:
#@title install packages + repos

# packages
!pip install matplotlib
!pip install torch
!pip install zarr

# repos
!pip install git+https://github.com/funkey/gunpowder.git
!pip install git+https://github.com/funkelab/funlib.learn.torch.git
!pip install git+https://github.com/funkey/waterz.git

In [ ]:
#@title import packages

import gunpowder as gp
import h5py
import io
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import requests
import torch
import zarr

from funlib.learn.torch.models import UNet, ConvPass
from gunpowder.torch import Predict

%matplotlib inline
logging.basicConfig(level=logging.INFO)

In [ ]:
# so we can load a presaved checkpoint in a subsequent cell
!git clone https://github.com/funkelab/lsd.git
!cd lsd && git checkout tutorial

In [ ]:
#@title utility function to view labels

# matplotlib uses a default shader
# we need to recolor as unique objects

def create_lut(labels):

    max_label = np.max(labels)

    lut = np.random.randint(
            low=0,
            high=255,
            size=(int(max_label + 1), 3),
            dtype=np.uint64)

    lut = np.append(
            lut,
            np.zeros(
                (int(max_label + 1), 1),
                dtype=np.uint8) + 255,
            axis=1)

    lut[0] = 0
    colored_labels = lut[labels]

    return colored_labels

In [ ]:
#@title utility  function to download / save data as zarr
def create_data(
    url, 
    name, 
    offset, 
    resolution,
    sections=None,
    squeeze=True):

  in_f = h5py.File(io.BytesIO(requests.get(url).content), 'r')

  raw = in_f['volumes/raw']
  labels = in_f['volumes/labels/neuron_ids']
  
  f = zarr.open(name, 'a')

  if sections is None:
    sections=range(raw.shape[0]-1)

  for i, r in enumerate(sections):

    print(f'Writing data for section {r}')

    raw_slice = raw[r:r+1,:,:]
    labels_slice = labels[r:r+1,:,:]

    if squeeze:
      raw_slice = np.squeeze(raw_slice)
      labels_slice = np.squeeze(labels_slice)

    f[f'raw/{i}'] = raw_slice
    f[f'labels/{i}'] = labels_slice

    f[f'raw/{i}'].attrs['offset'] = offset
    f[f'raw/{i}'].attrs['resolution'] = resolution

    f[f'labels/{i}'].attrs['offset'] = offset
    f[f'labels/{i}'].attrs['resolution'] = resolution

In [ ]:
# fetch a random section

create_data(
    'https://cremi.org/static/data/sample_A_20160501.hdf',
    'testing_data.zarr',
    offset=[0,0],
    resolution=[4,4],
    sections=random.sample(range(0,124),1),
    squeeze=False)

In [ ]:
voxel_size = gp.Coordinate((4, 4))

input_shape = gp.Coordinate((164, 164))
output_shape = gp.Coordinate((124, 124))

input_size = input_shape * voxel_size
output_size = output_shape * voxel_size

# total roi of image to predict on
total_roi = gp.Coordinate((1250,1250))*voxel_size

num_fmaps=12

In [ ]:
#@title create mtlsd model

class MtlsdModel(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.unet = UNet(
            in_channels=1,
            num_fmaps=num_fmaps,
            fmap_inc_factor=5,
            downsample_factors=[
                [2, 2],
                [2, 2]],
            kernel_size_down=[
                [[3, 3], [3, 3]],
                [[3, 3], [3, 3]],
                [[3, 3], [3, 3]]],
            kernel_size_up=[
                [[3, 3], [3, 3]],
                [[3, 3], [3, 3]]])

        self.lsd_head = ConvPass(num_fmaps, 6, [[1, 1]], activation='Sigmoid')
        self.aff_head = ConvPass(num_fmaps, 2, [[1, 1]], activation='Sigmoid')

    def forward(self, input):

        z = self.unet(input)
        lsds = self.lsd_head(z)
        affs = self.aff_head(z)

        return lsds, affs

class WeightedMSELoss(torch.nn.MSELoss):

    def __init__(self):
        super(WeightedMSELoss, self).__init__()

    def forward(self, lsds_prediction, lsds_target, lsds_weights, affs_prediction, affs_target, affs_weights,):

        loss1 = super(WeightedMSELoss, self).forward(
                lsds_prediction*lsds_weights,
                lsds_target*lsds_weights)

        loss2 = super(WeightedMSELoss, self).forward(
            affs_prediction*affs_weights,
            affs_target*affs_weights)
        
        return loss1 + loss2

In [ ]:
def predict(
    checkpoint,
    raw_file,
    raw_dataset,
    out_file,
    out_lsds,
    out_affs):
  
  raw = gp.ArrayKey('RAW')
  pred_lsds = gp.ArrayKey('PRED_LSDS')
  pred_affs = gp.ArrayKey('PRED_AFFS')

  if os.path.exists(out_file):
    mode='r+'
  else:
    mode='w'

  of = zarr.open(out_file, mode=mode)
  rd = zarr.open(raw_file)[raw_dataset]

  # create output datasets to write to
  for ds in [out_lsds, out_affs]:
      if ds not in of:
          if 'lsd' in ds:
              dims = 6
          else:
              dims = 2
          out_ds = of.create_dataset(
                  ds,
                  shape= (dims,) + np.squeeze(rd).shape,
                  dtype=np.float32)
          out_ds.attrs['resolution'] = voxel_size

  scan_request = gp.BatchRequest()

  scan_request.add(raw, input_size)
  scan_request.add(pred_lsds, output_size)
  scan_request.add(pred_affs, output_size)

  source = gp.ZarrSource(
              raw_file,
          {
              raw: raw_dataset
          },
          {
              raw: gp.ArraySpec(interpolatable=True)
          })

  model = MtlsdModel()

  # set model to eval mode
  model.eval()

  # add a predict node
  predict = gp.torch.Predict(
      model=model,
      checkpoint=checkpoint,
      inputs = {
                'input': raw
      },
      outputs = {
          0: pred_lsds,
          1: pred_affs})
  
  # this will scan in chunks equal to the input/output sizes of the respective arrays
  scan = gp.Scan(scan_request)

  # write out data
  write = gp.ZarrWrite(
      dataset_names={
          pred_lsds: out_lsds,
          pred_affs: out_affs},
      output_filename=out_file)
  
  pipeline = source
  pipeline += gp.Normalize(raw)
  pipeline += gp.Stack(1)
  pipeline += predict
  pipeline += scan
  pipeline += gp.Squeeze([pred_lsds, pred_affs])
  pipeline += write

  predict_request = gp.BatchRequest()

  # this lets us know to process the full image. we will scan over it until it is done
  predict_request.add(raw, total_roi)
  predict_request.add(pred_lsds, total_roi)
  predict_request.add(pred_affs, total_roi)

  with gp.build(pipeline):
      pipeline.request_batch(predict_request)

In [ ]:
checkpoint = 'lsd/lsd/tutorial/notebooks/model_checkpoint_50000' 
raw_file = 'testing_data.zarr'
raw_dataset = 'raw/0'
out_file = 'prediction.zarr'
out_lsds = 'pred_lsds/0'
out_affs = 'pred_affs/0'

predict(
    checkpoint,
    raw_file,
    raw_dataset,
    out_file,
    out_lsds,
    out_affs)

In [ ]:
test_f = zarr.open('testing_data.zarr')
predict_f = zarr.open('prediction.zarr')

raw = test_f['raw/0'][:]
labels = test_f['labels/0'][:]

pred_affs = predict_f['pred_affs/0'][:]
pred_lsds = predict_f['pred_lsds/0'][:]

fig, axes = plt.subplots(
            1,
            4,
            figsize=(20, 6),
            sharex=True,
            sharey=True,
            squeeze=False)

# view predictions (for lsds we will just view the mean offset component)
axes[0][0].imshow(np.squeeze(raw), cmap='gray')
axes[0][1].imshow(create_lut(np.squeeze(labels)))
axes[0][2].imshow(np.squeeze(pred_affs[0:1,:,:]), cmap='jet')
axes[0][3].imshow(np.squeeze(pred_lsds[0:1,:,:]), cmap='jet')
axes[0][3].imshow(np.squeeze(pred_lsds[1:2,:,:]), cmap='jet', alpha=0.5)

*  see how to generate a segmentation in **segment.ipynb**